# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 7 2022 3:54PM,245899,15,8497131,"Brookfield Pharmaceuticals, LLC",Released
1,Sep 7 2022 3:54PM,245899,15,8497132,"Brookfield Pharmaceuticals, LLC",Released
2,Sep 7 2022 3:54PM,245899,15,8497684,"Brookfield Pharmaceuticals, LLC",Released
3,Sep 7 2022 3:54PM,245899,15,8497685,"Brookfield Pharmaceuticals, LLC",Released
4,Sep 7 2022 3:54PM,245899,15,8497686,"Brookfield Pharmaceuticals, LLC",Released
5,Sep 7 2022 3:54PM,245899,15,8497687,"Brookfield Pharmaceuticals, LLC",Released
6,Sep 7 2022 3:54PM,245899,15,8497688,"Brookfield Pharmaceuticals, LLC",Released
7,Sep 7 2022 3:54PM,245899,15,8497689,"Brookfield Pharmaceuticals, LLC",Released
8,Sep 7 2022 3:50PM,245898,16,8493241,Sartorius Bioprocess Solutions,Released
9,Sep 7 2022 3:50PM,245898,16,8496466,Sartorius Bioprocess Solutions,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
44,245895,Released,1
45,245896,Released,3
46,245897,Released,1
47,245898,Released,4
48,245899,Released,8


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
245895,NaN,NaN,1.0
245896,NaN,NaN,3.0
245897,NaN,NaN,1.0
245898,NaN,NaN,4.0
245899,NaN,NaN,8.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
245714,22.0,2.0,2.0
245786,7.0,2.0,2.0
245807,0.0,0.0,19.0
245808,0.0,0.0,3.0
245810,0.0,0.0,8.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
245714,22,2,2
245786,7,2,2
245807,0,0,19
245808,0,0,3
245810,0,0,8


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,245714,22,2,2
1,245786,7,2,2
2,245807,0,0,19
3,245808,0,0,3
4,245810,0,0,8


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,245714,22,2,2
1,245786,7,2,2
2,245807,,,19
3,245808,,,3
4,245810,,,8


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 7 2022 3:54PM,245899,15,"Brookfield Pharmaceuticals, LLC"
8,Sep 7 2022 3:50PM,245898,16,Sartorius Bioprocess Solutions
12,Sep 7 2022 3:44PM,245897,22,"NBTY Global, Inc."
13,Sep 7 2022 3:40PM,245896,19,"AdvaGen Pharma, Ltd"
16,Sep 7 2022 3:39PM,245895,20,"HVL, LLC dba Atrium Innovations"
17,Sep 7 2022 3:31PM,245893,15,"Brookfield Pharmaceuticals, LLC"
25,Sep 7 2022 3:21PM,245892,10,Eywa Pharma Inc.
27,Sep 7 2022 3:01PM,245883,20,Alumier Labs Inc.
36,Sep 7 2022 2:58PM,245889,20,"Adare Pharmaceuticals, Inc."
37,Sep 7 2022 2:44PM,245887,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 7 2022 3:54PM,245899,15,"Brookfield Pharmaceuticals, LLC",,,8
1,Sep 7 2022 3:50PM,245898,16,Sartorius Bioprocess Solutions,,,4
2,Sep 7 2022 3:44PM,245897,22,"NBTY Global, Inc.",,,1
3,Sep 7 2022 3:40PM,245896,19,"AdvaGen Pharma, Ltd",,,3
4,Sep 7 2022 3:39PM,245895,20,"HVL, LLC dba Atrium Innovations",,,1
5,Sep 7 2022 3:31PM,245893,15,"Brookfield Pharmaceuticals, LLC",,1,7
6,Sep 7 2022 3:21PM,245892,10,Eywa Pharma Inc.,,,2
7,Sep 7 2022 3:01PM,245883,20,Alumier Labs Inc.,,1,8
8,Sep 7 2022 2:58PM,245889,20,"Adare Pharmaceuticals, Inc.",,,1
9,Sep 7 2022 2:44PM,245887,10,ISDIN Corporation,,1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 7 2022 3:54PM,245899,15,"Brookfield Pharmaceuticals, LLC",8,,
1,Sep 7 2022 3:50PM,245898,16,Sartorius Bioprocess Solutions,4,,
2,Sep 7 2022 3:44PM,245897,22,"NBTY Global, Inc.",1,,
3,Sep 7 2022 3:40PM,245896,19,"AdvaGen Pharma, Ltd",3,,
4,Sep 7 2022 3:39PM,245895,20,"HVL, LLC dba Atrium Innovations",1,,
5,Sep 7 2022 3:31PM,245893,15,"Brookfield Pharmaceuticals, LLC",7,1,
6,Sep 7 2022 3:21PM,245892,10,Eywa Pharma Inc.,2,,
7,Sep 7 2022 3:01PM,245883,20,Alumier Labs Inc.,8,1,
8,Sep 7 2022 2:58PM,245889,20,"Adare Pharmaceuticals, Inc.",1,,
9,Sep 7 2022 2:44PM,245887,10,ISDIN Corporation,,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 7 2022 3:54PM,245899,15,"Brookfield Pharmaceuticals, LLC",8,,
1,Sep 7 2022 3:50PM,245898,16,Sartorius Bioprocess Solutions,4,,
2,Sep 7 2022 3:44PM,245897,22,"NBTY Global, Inc.",1,,
3,Sep 7 2022 3:40PM,245896,19,"AdvaGen Pharma, Ltd",3,,
4,Sep 7 2022 3:39PM,245895,20,"HVL, LLC dba Atrium Innovations",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 7 2022 3:54PM,245899,15,"Brookfield Pharmaceuticals, LLC",8.0,NaN,NaN
1,Sep 7 2022 3:50PM,245898,16,Sartorius Bioprocess Solutions,4.0,NaN,NaN
2,Sep 7 2022 3:44PM,245897,22,"NBTY Global, Inc.",1.0,NaN,NaN
3,Sep 7 2022 3:40PM,245896,19,"AdvaGen Pharma, Ltd",3.0,NaN,NaN
4,Sep 7 2022 3:39PM,245895,20,"HVL, LLC dba Atrium Innovations",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 7 2022 3:54PM,245899,15,"Brookfield Pharmaceuticals, LLC",8.0,0.0,0.0
1,Sep 7 2022 3:50PM,245898,16,Sartorius Bioprocess Solutions,4.0,0.0,0.0
2,Sep 7 2022 3:44PM,245897,22,"NBTY Global, Inc.",1.0,0.0,0.0
3,Sep 7 2022 3:40PM,245896,19,"AdvaGen Pharma, Ltd",3.0,0.0,0.0
4,Sep 7 2022 3:39PM,245895,20,"HVL, LLC dba Atrium Innovations",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4916911,245.0,15.0,7.0
12,245837,1.0,0.0,0.0
15,1475006,85.0,10.0,58.0
16,491743,6.0,0.0,0.0
19,491771,3.0,8.0,0.0
20,1229390,24.0,11.0,9.0
22,245897,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4916911,245.0,15.0,7.0
1,12,245837,1.0,0.0,0.0
2,15,1475006,85.0,10.0,58.0
3,16,491743,6.0,0.0,0.0
4,19,491771,3.0,8.0,0.0
5,20,1229390,24.0,11.0,9.0
6,22,245897,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,245.0,15.0,7.0
1,12,1.0,0.0,0.0
2,15,85.0,10.0,58.0
3,16,6.0,0.0,0.0
4,19,3.0,8.0,0.0
5,20,24.0,11.0,9.0
6,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,245.0
1,12,Released,1.0
2,15,Released,85.0
3,16,Released,6.0
4,19,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,22
Status,,,,,,,
Completed,7.0,0.0,58.0,0.0,0.0,9.0,0.0
Executing,15.0,0.0,10.0,0.0,8.0,11.0,0.0
Released,245.0,1.0,85.0,6.0,3.0,24.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,22
0,Completed,7.0,0.0,58.0,0.0,0.0,9.0,0.0
1,Executing,15.0,0.0,10.0,0.0,8.0,11.0,0.0
2,Released,245.0,1.0,85.0,6.0,3.0,24.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,22
0,Completed,7.0,0.0,58.0,0.0,0.0,9.0,0.0
1,Executing,15.0,0.0,10.0,0.0,8.0,11.0,0.0
2,Released,245.0,1.0,85.0,6.0,3.0,24.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()